In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

from gcnn import GCNN
aspects = ['population', 'intervention', 'outcome']

Using TensorFlow backend.


[]


In [2]:
vec = cPickle.load(open('../data/vectorizers/allfields_with_embedding_5000.p', 'rb'))

In [3]:
index = vec.index
C = {'inputs' : ['abstract', 'population', 'intervention', 'outcome']}
for input in C['inputs'] :
    input_range = index[input]
    C[input] = vec.X[input_range[0]:input_range[1]]

In [11]:
model = load_model('../store/weights/single_setup/SatDec216:57:342017final1/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b : a, '<lambda>' : lambda a, b : a})
#results_folder = '../store/results/single_message_ThuOct1919:10:492017/'

In [12]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pool_'+aspect[0].upper()).inputs, 
                          [model.get_layer('pool_'+aspect[0].upper()).layers[16].output])

In [56]:
filters_pico = {}
sample_length = 500
sample = np.random.choice(C['abstract'].shape[0], sample_length, replace=False)
for input in C['inputs'] :
    for aspect in aspects :
        val = model_c1[aspect]([C[input][sample]])[0][:, :, 0]
        val = np.dstack((C[input][sample], val))
        filters_pico[(aspect, input)] = val#model_c1[aspect]([C[input][sample]])[0]

In [12]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[(aspect, 'abstract')]
    word_activations = np.zeros((vec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((C['abstract'][sample], filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words[aspect] = word_activations

In [14]:
top_words_df = pd.DataFrame()
top_words_size = 10
for aspect in aspects :
    popwords = activated_words[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    top_words_df[aspect] = top_words
top_words_df

,population,intervention,outcome
0,cancer,alfa,bacteriological
1,elective,desmopressin,classic
2,falciparum,educational,fibrosis
3,leg,enuresis,nocturnal
4,maintenance,epoetin,outpatients
5,outpatients,favoring,polycystic
6,plasmodium,fluticasone,probing
7,singleton,palivizumab,progression-free
8,undergoing,qqq,uterine
9,venous,reminder,visual


<b>Evaluating Filters on Cohen Data</b>

In [6]:
df = pd.read_csv('../data/files/test_cohen_dedup.csv')
cohenvec = cPickle.load(open('../data/vectorizers/cohendata_dedup_5000.p', 'rb'))
#cohenvec = cPickle.load(open('../data/vectorizers/decision_aids_vec_5000.p', 'rb'))
#cohenvec =cPickle.load(open('../../pubmed/pubmed_data_vec_5000_dedup_n.p'))

In [14]:
pool_cdsr = {}
X_abs = cohenvec.X
from sklearn.preprocessing import normalize


for aspect in aspects :
    pool_cdsr[aspect] = []
    bs = 1000
    for i in range(int(np.floor(len(X_abs)/bs)) + 1) :
        val = model_c1[aspect]([X_abs[i*bs:(i+1)*bs]])[0][:, :, 0]
        val = np.dstack((X_abs[i*bs:(i+1)*bs], val))
        pool_cdsr[aspect].append(val)
    pool_cdsr[aspect] = np.concatenate(pool_cdsr[aspect])
    #pool_cdsr[aspect] = softmax(pool_cdsr[aspect][:,:,0], axis=1)[:,:,np.newaxis]

filter_aspect = {}
for aspect in aspects :
    filter_aspect[aspect] = pool_cdsr[aspect]

In [6]:
activated_words_cohen = {}
for aspect in aspects :
    filters = filter_aspect[aspect]
    word_activations = np.zeros((cohenvec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((cohenvec.X, filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words_cohen[aspect] = word_activations

In [7]:
top_words_df_cohen = pd.DataFrame()
top_words_size = 10
for aspect in aspects :
    popwords = activated_words_cohen[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    print [str(v) for k, v in cohenvec.idx2word.items() if k in top_words]
    top_words = pd.Series([v for k, v in cohenvec.idx2word.items() if k in top_words])
    top_words_df_cohen[aspect] = top_words
top_words_df_cohen

['york', 'young', 'younger', 'youth', 'youths', 'yr', 'yrs', 'z', 'zealand', 'zero']
['york', 'young', 'younger', 'youth', 'youths', 'yr', 'yrs', 'z', 'zealand', 'zero']
['york', 'young', 'younger', 'youth', 'youths', 'yr', 'yrs', 'z', 'zealand', 'zero']


,population,intervention,outcome
0,york,york,york
1,young,young,young
2,younger,younger,younger
3,youth,youth,youth
4,youths,youths,youths
5,yr,yr,yr
6,yrs,yrs,yrs
7,z,z,z
8,zealand,zealand,zealand
9,zero,zero,zero


In [52]:
summed = {}
for aspect in aspects :
    summed[aspect] = filter_aspect[aspect].sum(axis=-1)

In [53]:
X_words = map(lambda s : [cohenvec.idx2word[w] for w in s], cohenvec.X)

In [19]:
s = 1127
#14, 20, 30, 50, 1000, 300, 600, 800, 900, 1200, 850
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float':float_formatter})
gates = {}
for aspect in aspects :
    gate0 = filter_aspect[aspect][s, :, 1]
    nz = sum([1 for i in filter_aspect[aspect][s, :, 0] if i == 0])
    gate0 = np.convolve(gate0, [1./3 for _ in range(3)], mode='same')
    gate0 = (gate0 - gate0[nz:].mean(axis=0))
    gate0 = gate0.clip(0, 1)
    gate0 = gate0 / gate0.max(axis=0)

    gates[aspect] = gate0
    
#gates_c = np.vstack()
gates4 = np.vstack([gates[aspect] for aspect in aspects])
gates5 = gates4.argmax(axis=0)#softmax(gates4*10)#/gates4.max(axis=0)

X_words = [vec.idx2word[int(i)] for i in filter_aspect[aspect][s, :, 0]]
for aspect in aspects :
    print aspect
    for i, j in zip(X_words, gates[aspect]) :
        if i != '[0]' :
            print color(i, bg=[255, int((1-j)*255), int((1-j)*255)]),
#     for i, j in zip(X_words, gates[aspect]) :
#         if i != '[0]' :
#             print '{\setlength{\\fboxsep}{0pt}\colorbox{green!' +str(int(j*100))+ '!white}{' + i + '}}',
#     print ""

print ""
c=['red','green','blue']
for i, l, j in zip(X_words, gates4.T, gates5.T) :
    #r,g,b = j
    if i != '[0]' :
        print color(i, bg=c[j]),#[int((1-g-b))*255, int((1-r-b)*255), int((1-r-g)*255)]),
# for i, l, j in zip(X_words, gates4.T, gates5.T) :
#     if i != '[0]' :
#         print '{\setlength{\\fboxsep}{0pt}\colorbox{'+c[j]+'!' +str(int(l[j]*100))+ '!white}{' + i + '}}',
# print ""



population
this article aims to review lessons unk about lipid lowering and unk in the past unk and to unk what unk the future may unk . results from a series of unk clinical trials confirm that unk significantly reduce cardiovascular morbidity and mortality in patients with and without previous coronary artery disease . the potential of this drug class has yet to be fully explored . studies currently under way will unk many of the unk questions . intervention
this article aims to review lessons unk about lipid lowering and unk in the past unk and to unk what unk the future may unk . results from a series of unk clinical trials confirm that unk significantly reduce cardiovascular morbidity and mortality in patients with and without previous coronary artery disease . the potential of this drug class has yet to be fully explored . studies currently under way will unk many of the unk questions . outcome
this article aims to review lessons unk about lipid lowering and unk in the past unk a

In [9]:
from colors import color

In [59]:
C['abstract'][sample].clip(0, 1).sum(axis=-1).argsort()[:20]

array([234, 393, 490, 347, 181, 300, 156, 313, 417, 220, 124, 130, 169,
       145, 244, 148, 262,  58, 476, 298])

In [70]:
def softmax(X, theta = 1.0, axis = None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats. 
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the 
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter, 
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)

    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p